In [1]:
# data_process is used to generate raw data and perform feature engineering with the data
#%run data_process
%run recommendation_engine_func.ipynb

In [2]:
import sys,os
print(sys.version)
print(os.getcwd())

3.5.2 |Anaconda 4.2.0 (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
/Users/zishuoli/Downloads/bigdata/final


In [3]:
'''import basic package'''
import pandas as pd
import numpy as np


'''import deep learning package'''
import keras as k
import tensorflow as tf


import requests
import json

from IPython.display import Image
from IPython.display import display
from IPython.display import HTML

import urllib.request

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image as kimage


Using TensorFlow backend.


# SVD Recommendation Part

In [39]:
rating_matrix = np.load('rating_matrix.npy')
movieId_vs_tmdbId = np.load('movieId_vs_tmdbId.npy').item()

In [43]:
items, items_rating = svd_recommend(rating_matrix,movieId_vs_tmdbId,10)

In [44]:
movie_list_svd = items[0]
url_svd = [get_poster(x) for x in items[0]]

display_images(url_svd)

# Cosine similarity recommendation part

In [45]:
'''
split train & test
'''
train_matrix = rating_matrix.copy()
test_matrix = np.zeros(rating_matrix.shape)

for i in range(rating_matrix.shape[0]):
    rating_idx = np.random.choice(
        rating_matrix[i, :].nonzero()[0], 
        size=10, 
        replace=True)
    train_matrix[i, rating_idx] = 0.0
    test_matrix[i, rating_idx] = rating_matrix[i, rating_idx]


In [46]:
similarity_user = cosine_similarity_matrix(train_matrix)
print('similarity user matrix shape:',similarity_user.shape)
similarity_movie = cosine_similarity_matrix(train_matrix.T)
print('similarity movie matrix shape:',similarity_movie.shape)


similarity user matrix shape: (671, 671)
similarity movie matrix shape: (9000, 9000)


In [47]:
prediction = prediction_(similarity_user,rating_matrix)
prediction = prediction[test_matrix.nonzero()].flatten()
test_vector = test_matrix[test_matrix.nonzero()].flatten()
print('real_value:',test_vector)
print('prediction:',prediction)

real_value: [ 2.5  3.   2.  ...,  3.5  4.5  4. ]
prediction: [ 0.54373842  0.67968005  1.02033544 ...,  1.09396875  1.18874884
  0.58826262]


In [48]:
mse = error(prediction=prediction,real_value=test_vector)
print('mse for prediction using cosine similarity',mse)

mse for prediction using cosine similarity 8.94021


In [49]:
movie_list_cosine = similar_select(similarity_movie,movieId_vs_tmdbId)
url_list_cosine = [get_poster(x) for x in movie_list_cosine]

display_images(url_list_cosine)

# Deep Learning Part

In [50]:
feature_matrix = np.load('feature_matrix.npy')
movie_list  = np.load('movie_list.npy')

In [51]:
similarity_movie = cosine_similarity_matrix(feature_matrix)

In [55]:
show_result(similarity_movie,movieId_vs_tmdbId,movie_list,target_id=4,n=5)

In [11]:
show_result(similarity_movie,movie_list,5)

In [12]:
show_result(similarity_movie,movie_list,7)

In [14]:
show_result(similarity_movie,movie_list,8)

In [15]:
show_result(similarity_movie,movie_list,9)

In [16]:
show_result(similarity_movie,movie_list,10)